`{r setup, include=FALSE} knitr::opts_chunk$set(echo = TRUE)`

## R Markdown

``` {r}
library(plotly)
library(dplyr)
library(corrplot)
library(ggplot2)
library(factoextra)
library(FactoMineR)
library(gridExtra)
library(caret)
library(glmnet)
library(MASS)
library(e1071)
library(rpart)
library(rpart.plot)
library(partykit)
library(randomForest)
library(gbm)
library(neuralnet)
library(ggRandomForests)
library(xgboost)
library(nnet)
```

``` {r}
#QUESTION 1


data <- read.csv("global-data-on-sustainable-energy (1).csv", header = TRUE)
head(data)
```

``` {r}
#On affiche les types des variables
str(data)
```

``` {r}
summary(data)
```

``` {r}
#On transforme la variable Year en qualitative
data$Year <- as.factor(data$Year)
class(data$Year)
```

``` {r}
#On transforme la densité en variable numérique
data$Density.n.P.Km2.<-as.numeric(gsub(",","",data$Density.n.P.Km2.))
class(data$Density.n.P.Km2.)

```

``` {r}
#On vérifie qu'on a les bons types 
str(data)
```

``` {r}
#QUESTION 2


#Afficher le nombre de valeurs manquantes par variables

données_manquantes <- colSums(is.na(data))
print(données_manquantes)
```

``` {r}

fig <- plot_ly(x = names(données_manquantes), y = données_manquantes, type = "bar", 
               name = "Valeurs manquantes par variables", 
               marker = list(color = "purple"),
               xlab = "Variables", ylab = "Nombre de valeurs manquantes",
               title = "Valeurs manquantes par variables")

fig
```

``` {r}
#On récupère le nom complet des variables
names(data)
```

``` {r}
#On supprime les variables comportant un taux de données manquantes très important

data2 <- subset(data, select = -c(Renewable.electricity.generating.capacity.per.capita, Financial.flows.to.developing.countries..US..., Renewables....equivalent.primary.energy.))
```

``` {r}
#On vérifie que data2 a bien les 3 variables précédentes en moins

names(data2)
```

``` {r}
#QUESTION 3

# On créée un nouveau jeu de données comportant seulement les individus sans valeurs manquantes

data_sans_val_manquantes <- data2[complete.cases(data2), ]
```

``` {r}
#On vérifie qu'on a bien 2868 observations dans le jeu de données complet

nrow(data_sans_val_manquantes)
```

``` {r}
#QUESTION 4

library(ggplot2)

ggplot(data_sans_val_manquantes, aes(y = `Access.to.electricity....of.population.`)) +
  geom_boxplot(fill = "blue") +
  labs(title = "Boîte à moustaches pour Access-to-electricity")
```

``` {r}

data3 <- data_sans_val_manquantes

# Autre type d'analyse descriptive de variable qualitative

ggplot(data3, aes(x = `Electricity.from.fossil.fuels..TWh.`)) +
  geom_histogram(binwidth = 50, fill = "blue", color = "black") +
  labs(title = "Electricity-from-fossil-fuels",
       x = "Electricity-from-fossil-fuels (TWh)",
       y = "Fréquence") +
  theme_minimal()

# Nous remarquons que cette variable possède bcp de données entre 0 et 500 TWh
```

``` {r}
#ggplot(data3, aes(x = `Electricity.from.fossil.fuels..TWh.`)) +
 # geom_density(fill = "skyblue", alpha = 0.5) +
  #labs(title = "Densité de Electricity-from-fossil-fuels")
```

``` {r}
ggplot(data3, aes(x = `Access.to.clean.fuels.for.cooking`)) +
  geom_density(fill = "blue", alpha = 0.5) +
  labs(title = "Densité de Access.to.clean.fuels.for.cooking")
```

``` {r}

# Tracer le graphique de densité
ggplot(data3, aes(x=`Primary.energy.consumption.per.capita..kWh.person.`)) +
  geom_density(fill = "blue", alpha = 0.5) +
  labs(title = "The density of Primary energy consumption per capita (kWh/person)")
```

``` {r}
#On applique une transformation logarithmique à cette variable

data3$Primary_energy_consumption_per_capita_new <- log(data3$Primary.energy.consumption.per.capita..kWh.person.)

#On supprime la variable sans transformation

data3 <- subset(data3, select = -c(Primary.energy.consumption.per.capita..kWh.person.))


```

``` {r}
#On affiche la nouvelle densité :
ggplot(data3, aes(x=`Primary_energy_consumption_per_capita_new`)) +
  geom_density(fill = "blue", alpha = 0.5) +
  labs(title = "The density of Primary energy consumption per capita new (kWh/person)")
```

``` {r}
#Nous avons bien effectué notre transformation logarithmique sur la variable "Primary.energy.consumption.per.capita..kWh.person.".
```

``` {r}
#QUESTION 5


#On renomme la colonne "Value_co2_emissions_kt_by_country_new" en "CO2" 
data3 <- data3 %>%
  rename(CO2 = Value_co2_emissions_kt_by_country)

```

``` {r}
#On vérifie :
names(data3)
```

``` {r}

#Nous pouvons observer les émissions de CO2 par pays

ggplot(data3, aes(x = Entity, y = CO2)) +
  geom_bar(stat = "identity", fill = "blue") +
  labs(title = "Émissions de CO2 par pays",
       x = "Pays",
       y = "Émissions de CO2 (en tonnes)") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) 

#Légende difficile à lire mais on voit une hétérogénéité entre les pays
```

``` {r}
#On veut mtn voir les 5 pays les plus emetteurs en CO2:


#On calcule les émissions maximales de CO2 par pays
CO2_max <- data3 %>%
  group_by(Entity) %>%
  summarise(CO2 = max(CO2)) %>%
  arrange(desc(CO2)) %>%
  head(5)

ggplot(CO2_max, aes(x = reorder(Entity, CO2), y = CO2, fill = CO2)) +
  geom_bar(stat = "identity") +
  labs(title = "Top 5 des pays ayant émis le plus de CO2",
       x = "Pays",
       y = "Émissions de CO2",
       fill = "Émissions de CO2") +
  scale_fill_gradient(low = "blue", high = "yellow") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))  # Rotation des noms de pays sur l'axe x
```

``` {r}
#QUESTION 6
str(data3)
names(data3)
```

``` {r}


# On renomme les colonnes du dataframe 3
data4 <- data3 %>%
  rename(Access_to_electricity = Access.to.electricity....of.population.,
         Access_to_clean_fuels_for_cooking = Access.to.clean.fuels.for.cooking,
         Renewable_energy_share_in_the_total_final_energy_consumption = Renewable.energy.share.in.the.total.final.energy.consumption....,
         Electricity_from_fossil_fuels = Electricity.from.fossil.fuels..TWh.,
         Electricity_from_nuclear = Electricity.from.nuclear..TWh.,
         Electricity_from_renewables = Electricity.from.renewables..TWh.,
         Low_carbon_electricity = Low.carbon.electricity....electricity.,
         Energy_intensity_level_of_primary_energy = Energy.intensity.level.of.primary.energy..MJ..2017.PPP.GDP.,
         Density_n_P_Km2 = Density.n.P.Km2.,
         Land_Area = Land.Area.Km2.)

names(data4)
```

``` {r}
# On convertit la variable "Land-Area" qui est en entier en réel
data4 <- data4 %>%
  mutate(Land_Area = as.numeric(Land_Area))
str(data4)
```

``` {r}
names(data4)


cols <- c('Access_to_electricity',
          'Access_to_clean_fuels_for_cooking',
          'Renewable_energy_share_in_the_total_final_energy_consumption',
          'Electricity_from_fossil_fuels',
          'Electricity_from_nuclear',
          'Electricity_from_renewables',
          'Low_carbon_electricity',
          'Primary_energy_consumption_per_capita_new',
          'Energy_intensity_level_of_primary_energy',
          'CO2',
          'gdp_growth',
          'gdp_per_capita',
          'Density_n_P_Km2',
          'Land_Area',
          'Latitude',
          'Longitude')


correlation_matrix <- cor(data4[cols])
dim(data4)

corrplot(correlation_matrix, 
          method = "color", 
          type = "upper", 
          tl.col = "black", 
          tl.srt = 45, 
          tl.cex = 0.7,
          diag = FALSE,  
          addCoef.col = "black",  
          number.cex = 0.4,  
          tl.pos = "lt")
```

``` {r}
corrplot(correlation_matrix, 
          method = "ellipse",  # Utiliser la méthode ellipse
          tl.pos = "lt", type = "upper", 
          tl.col = "black", 
          tl.srt = 10, 
          tl.cex = 0.6)
```

``` {r}
#On trace un scatterplot pour toutes les variables de corrélation

#pairs(data4[, c('Access_to_electricity', 'Access_to_clean_fuels_for_cooking', 'Renewable_energy_share_in_the_total_final_energy_consumption', 'Electricity_from_fossil_fuels', 'Electricity_from_nuclear', 'Electricity_from_renewables', 'Low_carbon_electricity', 'Primary_energy_consumption_per_capita_new', 'Energy_intensity_level_of_primary_energy', 'CO2', 'gdp_growth', 'gdp_per_capita', 'Density_n_P_Km2', 'Land_Area', 'Latitude', 'Longitude')], main = "Scatterplot pour les variables de corrélation")

#On voit pas grand chose, on va essayer de tracer plutôt un scatterplot pour les deux variables les plus corrélées
```

``` {r}
#On veut voir quelles sont les paires de variables les plus corrélées
paires_plus_cor <- which(correlation_matrix == max(correlation_matrix[lower.tri(correlation_matrix)], na.rm = TRUE), arr.ind = TRUE)

#On affiche ces variables
var_plus_cor <- colnames(correlation_matrix)[paires_plus_cor[, 1]]
var_plus_cor
```

``` {r}

#On affiche un nuage de points pour visualiser les relations entre les paires de variables les plus corrélées
scatterplot <- ggplot(data4, aes(x = Electricity_from_fossil_fuels, y = CO2)) +
  geom_point() +  
  labs(title = "Relation entre les variables les plus corrélées : C02 et Electricity_from_fossil_fuels",
       x = "Electricity_from_fossil_fuels",
       y = "CO2")

scatterplot
```

``` {r}
#QUESTION 7
library(ggbiplot)


#On créée un nouveau dataframe pour garder seulement les variables quantitatives

data5 <- subset(data4, select = -c(Entity, Year))

#On fait une ACP
#On met "true" pour standardiser les données

pca <- PCA(data4,quali.sup=c(1,2), scale. = TRUE,graph=FALSE)
summary(pca)

fviz_pca_ind(pca,geom=c("point"), habillage = data4[,2])
```

En 2 dimensions on explique 50% de la variance, on distingue bcp
d’outliers

``` {r}
#On affiche la qualité de représentation des variables : comment les axes permettent d'expliquer les variables
q1=fviz_pca_var(pca, col.var="cos2",repel=TRUE,gradient.cols=c("#00afbb","red","yellow"))
q1

```

``` {r}
#On affiche la contribution des variables aux axes

c1=fviz_pca_var(pca,col.var="contrib",repel=TRUE,gradient.cols=c("#00afbb","red","yellow"))
c1
```

``` {r}
q2=fviz_pca_ind(pca, col.ind="cos2",geom=c("point"),gradient.cols=c("#00afbb","red","yellow"))
q2
```

Les individus proches de 0 sont mal expliqués

``` {r}
c2=fviz_pca_ind(pca, col.ind="contrib",geom=c("point"),gradient.cols=c("#00afbb","red","yellow"))
c2
```

``` {r}

fviz_eig(pca)
```

``` {r}
#Plus la proportion de variance expliquée est élevée pour une composante principale donnée, plus cette composante principale est importante pour représenter la structure des données : la 1ère composante est celle qui est la plus importante ici
```

``` {r}
#QUESTION 8

#Ici on doit travailler avec data4 pour avoir les variables qualitatives

#On affiche une dépendance linéaire entre les variables C02 et Year

ggplot(data4, aes(x = Year, y = CO2)) +
  geom_point() +  
 labs(title = "Dépendance entre l'année et les émissions de CO2",
       x = "Année",
       y = "Émissions de CO2 (en kt)")
```

``` {r}
#On fait la même chose mais avec un graphique intéractif pour avoir la valeur exacte de CO2 en chaque point
library(plotly)
plot_ly(data = data4, x = ~Year, y = ~CO2, type = "scatter", mode = "markers")
```

``` {r}

#On trace un graphique en violon des émissions de CO2 par année

ggplot(data4, aes(x = Year, y = CO2)) +
  geom_violin(fill = "purple") +
  labs(title = "Emissions de CO2 par année",
       x = "Année",
       y = "Émissions de CO2 (en kt)")


```

``` {r}
#Diagramme en boîte des émissions de CO2 par année

ggplot(data4, aes(x = Year, y = CO2)) +
  geom_boxplot(fill = "purple") +
  labs(title = "Diagramme en boite des émissions de CO2 par année",
       x = "Année",
       y = "Émissions de CO2 (en kt)")
```

``` {r}

#On créé un df avec les valeurs max de CO2 par année

CO2_max_year <- aggregate(data4$CO2, by=list(Year=data4$Year), FUN=max)
CO2_max_year <- CO2_max_year[order(CO2_max_year$x),]
CO2_max_year20 <- head(CO2_max_year, 20)
names(CO2_max_year20) <- c("Year", "CO2")

# Création du graphique à barres
fig <- ggplot(CO2_max_year20, aes(x = Year, y = CO2, fill = CO2)) +
  geom_bar(stat = "identity") +
  labs(x = "Year", y = "Emissions CO2", title = "Quantité de CO2 maximale en fonction des années") +
  theme_minimal()

print(fig )
```

``` {r}
#Le max des CO2 n'est pas judicieux car on a bcp de outliers

#On créé un df avec les valeurs moyennes de CO2 par année

CO2_moy_year <- aggregate(data4$CO2, by=list(Year=data4$Year), FUN=mean)
CO2_moy_year <- CO2_moy_year[order(CO2_moy_year$x),]
CO2_moy_year20 <- head(CO2_moy_year, 20)
names(CO2_moy_year20) <- c("Year", "CO2")

# Création du graphique à barres
fig <- ggplot(CO2_moy_year20, aes(x = Year, y = CO2, fill = CO2)) +
  geom_bar(stat = "identity") +
  labs(x = "Year", y = "Emissions CO2", title = "Quantité de CO2 moyenne en fonction des années") +
  theme_minimal()

print(fig )
```

``` {r}
#Diagramme en boîte des émissions de CO2 par année
data_log <- data4
data_log <- data_log[, 11, drop = FALSE]  # Conserver la structure de dataframe
data_log[, 1] <- log(data_log[, 1])  # Appliquer log à la colonne extraite
data_log$Year <- data4$Year  # Réattribuer la colonne Year


ggplot(data_log, aes(x = Year, y = CO2)) +
  geom_boxplot(fill = "purple") +
  labs(title = "Diagramme en boite des émissions de CO2 par année",
       x = "Année",
       y = "Émissions de CO2 (en kt)")
```

``` {r}
#On va faire une ANOVA à 1 facteur pour analyser la dépendance d'une variable quantitative avec une variable qualitative

anova1 <- lm(CO2 ~ Year, data = data4)
summary(anova1)
```

``` {r}
#On créé un 2ème modèle pour comparer avec le précédent 
anova2 <- lm(CO2 ~ 1, data = data4)

#On compare le sous modèle avec le modèle1, on a une p-val>0.05, c'est sur qu'on accepte ce sous modèle.
anova(anova2,anova1)
```

``` {r}
#On vérifie ces résultats avec un test d'indépendance du chi2
chi2_test <- chisq.test(data4$CO2, data4$Year)
print(chi2_test)

#pval=0.14>0.05 : on rejette pas H0, CO2 et Year sont indépendants
```

``` {r}
#On se rend compte que sur les tests on peut enlever l'année alors que sur les graphes des quantités moyennes de CO2 on voit quand même une augmentation au fil des années

#On essaye donc de faire un modèle en fonction du pays et de l'année
anova3 <- lm(CO2 ~ Year+Entity, data = data4)
summary(anova3)
```

Le R^2 est de 0.93 donc modèle bcp mieux

``` {r}
#On fait mtn un sous modèle en fonction seulement du pays
anova4=lm(CO2 ~ Entity, data = data4)
#On compare ce sous modèle avec le précédent
anova(anova4,anova3)
```

On voit que pval=0.15\>0.05, donc on peut garder le sous modèle et donc
l’effet de l’année est en fait négligeable par rapport à celui du pays

``` {r}
#----------------------------------------------PARTIE 2--------------------------------------------------------------------




#QUESTION 1
data6 <- subset(data4, select =-Electricity_from_fossil_fuels)
#data6 <- data4 %>% 
#  select(-Electricity_from_fossil_fuels)
```

``` {r}
data6$Entity <- as.factor(data6$Entity)
class(data6$Entity)
```

``` {r}
names(data6) 

#On sélectionne les variables prédictives : toutes sauf la variable à prédire
#On enlève également les variables quantitatives
data7 <- subset(data6, select =-c(Entity,Year))

X <- subset(data7, select = -c(CO2))

#On garde la variable à prédire : CO2

y <- data7$CO2



#On divise les données en un ensemble d'apprentissage et un ensemble de test
set.seed(123)

#Ici on garde que 20% des données pour l'échantillon test

train_index <- createDataPartition(y, p = 0.8, list = FALSE)

#Données d'apprentissage :

train_data <- data7[train_index, ]

#Données de test :
test_data <- data7[-train_index, ]

# vérification
str(train_data)
str(test_data)


```

``` {r}
#Pour pouvoir comparer les modèles, on créé une fonction qui permet de tracer les résidus avec une échelle fixe
gplot.res <- function(x, y, titre = "titre"){
    ggplot(data.frame(x=x, y=y),aes(x,y))+
    geom_point(col = "blue")+
    ylab("Résidus")+ xlab("Valeurs prédites")+
    ggtitle(titre)+
    geom_hline(yintercept = 0,col="green")
}
```

``` {r}
#QUESTION 2



# 1) Regression linéaire


#Sans sélection de variables

lm_model_complet <- aov(CO2 ~ ., data = train_data)
res.lm <- lm_model_complet$residuals
fit.lm <- lm_model_complet$fitted.values
gplot.res(fit.lm,res.lm,"ANCOVA sans sélection de variables")
```

``` {r}

#Avec sélection de variables : on cherche les variables à garder

lm_model_AIC<- stepAIC(lm_model_complet, trace=FALSE)
#summary(lm_model_AIC)
res.lm.selec <- lm_model_AIC$residuals
fit.lm.selec <- lm_model_AIC$fitted.values
gplot.res(fit.lm.selec,res.lm.selec,"ANCOVA avec sélection de variables")
```

``` {r}

#Avec pénalisation (par exemple Lasso)

#On enlève la variable C02 et on transforme en matrice

x=as.matrix(train_data[-1])
y=train_data$CO2

#On fait avec validation croisée

lasso_model <- cv.glmnet(x, y, alpha = 1)
res.lasso <- lasso_model$residuals
fit.lasso <- lasso_model$fitted.values
gplot.res(fit.lasso,res.lasso,"ANCOVA avec Lasso")


#ou----------------------------------------------------------------------------------------------

x.mat <- model.matrix(CO2 ~ . - 1, data = train_data) #notre x.mat c'est récupéré juste la matrice de design dans notre modèle où on enlève un intercept, 
                                                     #cad qu'on va prendre 2 modalités pour jour, soit on a un intercept soit on prend toutes les modalités pour une var; 
                                                     #mais on garde toujours 4modalités/5 pour la station
reg.lasso <- glmnet(y = train_data$CO2, x = x.mat)
options(repr.plot.width = 12, repr.plot.height = 10)
plot(reg.lasso, xvar = "lambda", label = TRUE)
#legend("topright", 
       #legend = paste(1:ncol(x.mat), " - ", colnames(x.mat)))
```

``` {r}
#2) SVM

#Avec noyau linéaire

svm_model_linear <- svm(CO2 ~ ., data = train_data, kernel = "linear")

#Avec noyau gaussien 

svm_model_gaussien <- svm(CO2 ~ ., data = train_data, kernel = "radial")

#On fait SVM en régression sur le noyau par défaut, le "radial"-> Gaussien

svm_model0 = svm(CO2 ~ ., data = train_data)
set.seed(2021)
svm.reg.tune = tune.svm(CO2 ~ ., data = train_data, cost = c(1,10,100,1000), 
    gamma = seq(0.0, 0.3, by = 0.05))
plot(svm.reg.tune)


```

``` {r}
svm_reg = svm(CO2 ~ ., data = train_data, cost = svm.reg.tune$best.parameters$cost, 
    gamma = svm.reg.tune$best.parameters$gamma)
summary(svm_reg)
```

On voit que le modèle optimal est celui avec le noyau gaussien et un
cout de 3,5.

``` {r}
#Tracé des résidus
fit.svmr = svm_reg$fitted
res.svmr = fit.svmr - train_data[,"CO2"]

gplot.res(fit.svmr,res.svmr,titre="Residus SVM")


plot(res.svmr, main = "Residus SVM", ylim = c(min(res.svmr), max(res.svmr)))
```

``` {r}
#3) Arbre de décision

#Sélection des paramètres optimaux par validation croisée
tree_model <- rpart(CO2 ~ ., data = train_data, control=rpart.control(cp=c(0.00001,0.0001,0.001, 0.01, 0.1)))
options(repr.plot.width = 500, repr.plot.height = 500)
rpart.plot(tree_model)
```

``` {r}

#On créé une matrice contenant les prédictions de l'arbre de décision tree_model pour chaque observation du jeu de données
#Chaque ligne correspond à une observation et chaque colonne à une valeur de CP

xmat<-xpred.rpart(tree_model,xval=10) 

#Erreur de validation croisée pour chaque valeur de CP

CVerr<-apply((xmat-train_data[,"CO2"])^2,2,sum)

plotcp(tree_model)
```

``` {r}
#On trouve le cp qui minimise l'erreur pour construire l'arbre optimal
which.min(CVerr)
```

La taille de l’arbre qui minimise vaut 8

``` {r}
as.numeric(attributes(which.min(CVerr))$names)
tree_opt_model=rpart(CO2~.,data=train_data,control=rpart.control(cp=as.numeric(attributes(which.min(CVerr))$names)))
rpart.plot(tree_opt_model,type=5,extra=101)
```

``` {r}
plot(as.party(tree_opt_model), type="simple")
```

``` {r}
#Tracé des résidus 

fit.tree=predict(tree_opt_model)
res.tree=fit.tree-train_data[,"CO2"]
gplot.res(fit.tree,res.tree,"Residus de l'arbre optimal")
plot(res.tree, main = "Residus  de l'arbre optimal", ylim = c(min(res.tree), max(res.tree)))
```

``` {r}
#4) Forêt aléatoire


#Sélection des paramètres optimaux par validation croisée


rf_model <- randomForest(CO2 ~ ., data = train_data, ntree = 500)
```

``` {r}
#Tracé des résidus
fit.rf=rf_model$predicted
res.rf=fit.rf-train_data[,"CO2"]
gplot.res(fit.rf,res.rf,titre="Residus RandomForest")
```

``` {r}
#Tracé des résidus sur l'échantillon de test
fit.rf=rf_model$predicted
res.rf=fit.rf-test_data[,"CO2"]

gplot.res(fit.rf,res.rf,titre="Residus RandomForest")


plot(res.rf, main = "Residus RandomForest", ylim = c(min(res.rf), max(res.rf)))
```

``` {r}
#5) Boosting



#Sélection des paramètres optimaux par validation croisée

boost_model <- gbm(CO2 ~ ., data = train_data, distribution = "gaussian", n.trees = 1000, 
    cv.folds = 10, n.minobsinnode = 5, shrinkage = 0.03, verbose = FALSE)
plot(boost_model$cv.error, type = "l")


```

``` {r}
#Nombre optimal d'itérations par valiation croisée
best.iter=gbm.perf(boost_model,method="cv")
```

``` {r}
#On fait varier le coefficient de rétrécissement

#On vérifie donc qu'il n'y a pas de sur-ajustement : l'erreur reste stable autour du nombre d'arbres sélectionné et matérialisé par la ligne verticale

test=numeric()
for (i in 10:1000){
pred.test=predict(boost_model,newdata=test_data,n.trees=i)
err=sum((pred.test-test_data[,"CO2"])^2)/nrow(test_data)
test=c(test,err)}
plot(10:1000,test,type="l")
abline(v=best.iter)
```

``` {r}
#6) Réseau de neurones


# Sélection des paramètres optimaux par validation croisée
#reseau_model <- neuralnet(CO2 ~ ., data = train_data, hidden = c(5, 2))


# Entraînement du modèle de réseau de neurones pour la régression
# Assurez-vous d'adapter les noms des variables et le nom de l'ensemble de données en fonction de votre cas spécifique

res_neurones <- nnet(CO2 ~ ., data = train_data, size = 5, decay = 1, linout = TRUE, maxit = 500) 
```

``` {r,eval=f}
#On optimise les paramètres avec la validation croisée

plot(tune.nnet(CO2 ~ ., data = train_data, size = c(5, 10, 20), decay = c(1, 2, 3), maxit = 200, linout = TRUE))
plot(tune.nnet(CO2 ~ ., data = train_data, size = 5:20, decay = 1:10))
```

``` {r}
#On fait un modèle avec les paramètres optimisés 

res_neurones_opt=nnet(CO2~.,data=train_data,size=3,decay=2,linout=TRUE,maxit=200)

#On trace les résidus 
fit.nnetr=predict(res_neurones_opt,data=train_data)
res.nnetr=fit.nnetr-train_data[,"CO2"]
gplot.res(fit.nnetr,res.nnetr,titre="")
```

``` {r}
#QUESTION 3

#Prédiction sur l'ensemble de test
#Pour chaque modèle on va calculer l'erreur de généralisation


#Calculs des prédictions sur l'échantillon test pour tous les modèles optimaux

lm_AIC_pred <- predict(lm_model_AIC, newdata = test_data)
svm_pred <- predict(svm_reg, newdata = test_data)
tree_pred <- predict(tree_opt_model, newdata = test_data)
rf_pred <- predict(rf_model, newdata = test_data)
boost_pred <- predict(boost_model, newdata = test_data)

#Calculs des métriques de performance

lm_AIC_rmse <- RMSE(lm_AIC_pred, test_data$CO2)
svm_rmse <- RMSE(svm_pred, test_data$CO2)
tree_rmse<- RMSE(tree_pred, test_data$CO2)
rf_rmse <- RMSE(rf_pred, test_data$CO2)
boost_rmse <- RMSE(boost_pred, test_data$CO2)

#On affiche les erreurs de généralisation calculées

cat("RMSE pour le modèle de régression linéaire avec AIC :", lm_AIC_rmse, "\n")
cat("RMSE pour le modèle SVM :", svm_rmse, "\n")
cat("RMSE pour le modèle d'arbre :", tree_rmse, "\n")
cat("RMSE pour le modèle de forêt aléatoire :", rf_rmse, "\n")
cat("RMSE pour le modèle Boosting :", boost_rmse, "\n")
```

``` {r}


#On fait les prédictions des différents modèles
predictions <- data.frame(
  LM_AIC = lm_AIC_pred,
  SVM = svm_pred,
  Tree = tree_pred,
  Random_forest=rf_pred,
  Boost=boost_pred
)

predictions$True <- test_data$CO2


ggplot(predictions, aes(x = True)) +
  geom_point(aes(y = LM_AIC, color = "LM_AIC"), size = 2, alpha = 0.5) + 
  geom_point(aes(y = SVM, color = "SVM"), size = 2, alpha = 0.5) + 
  geom_point(aes(y = Tree, color = "Tree"), size = 2, alpha = 0.5) +
  geom_point(aes(y = Random_forest, color = "Random Forest"), size = 2, alpha = 0.5) +
  geom_point(aes(y = Boost, color = "Boosting"), size = 2, alpha = 0.5)+
  geom_abline(slope = 1, intercept = 0, linetype = "dashed") + 
  labs(title = "Comparaison des prédictions par rapport aux vraies valeurs",
       x = "Vraies valeurs de CO2",
       y = "Valeurs prédites",
       color = "Modèles") + 
  scale_color_manual(values = c("blue", "red", "green","pink","orange"), labels = c("LM_AIC", "SVM", "Tree","Random Forest", "Boosting")) + # 
  theme_minimal()

```

``` {r}
#QUESTION 4 

#On regarde l'importance des variables 


#Pour RandomForest:
importance <- round(importance(rf_model)[, 1], 2)
importance_sorted <- sort(importance, decreasing = TRUE)

#On affiche seulement les 10 premières variables les plus importantes

top_10_importance <- importance_sorted[1:10]
print(top_10_importance)

#On voit que pour Rf, c'est Electricity_from_renewables qui est la plus importante
varImpPlot(rf_model)

plot(gg_vimp(rf_model))


```

On a des résultats cohérents : les variables
Electricity_from_renewables, Land_area et Electricity_from_nuclear sont
très corrélées avec la variable C02 dans notre matrice de corrélation de
la partie 1. On voit que ce sont les 3 variables les plus importantes
pour le modèle de Random Forest.

``` {r}
#Pour les arbres

rpart.plot(tree_model)
```

Pour les arbres on voit que la variable la plus importante est
Electricity_from_renewables (car en haut de l’arbre), c’est donc
également cohérent.

``` {r}
#Pour la regression linéaire
#cat("Coeff de Lasso :", coefficients(lasso_model), "\n")

coefficients(lm_model_AIC)

which.max(coefficients(lm_model_AIC))
which.max(coefficients(lm_model_complet))

coefficients(lm_model_complet)
```

``` {r}

#Pour la regression linéaire on affiche les coeff des variables 

#Modele AIC

variable_names <- names(coefficients(lm_model_AIC))
coefficients <- coef(lm_model_AIC)

reg_aic_coefficients <- data.frame(
  Variables = variable_names,
  Coefficient = coefficients
)

#On enlève l'intercept pour le graphe
reg_aic_coefficients <- reg_aic_coefficients[reg_aic_coefficients$Variables != "(Intercept)", ]
ggplot(reg_aic_coefficients, aes(x = Variables, y = Coefficient)) +
  geom_bar(stat = "identity", fill = "skyblue", width = 0.5) +
  coord_flip() +
  labs(title = "Coeff pour le modèle AIC",
       x = "Variables",
       y = "Coefficient") +
  theme_minimal()
```

``` {r}
#Modele complet

variable_names <- names(coefficients(lm_model_complet))
coefficients <- coef(lm_model_complet)

reg_complet_coefficients <- data.frame(
  Variables = variable_names,
  Coefficient = coefficients
)

#On enlève l'intercept pour le graphe
reg_complet_coefficients <- reg_complet_coefficients[reg_complet_coefficients$Variables != "(Intercept)", ]
ggplot(reg_complet_coefficients, aes(x = Variables, y = Coefficient)) +
  geom_bar(stat = "identity", fill = "skyblue", width = 0.5) +
  coord_flip() +
  labs(title = "Coeff pour le modèle reg complet",
       x = "Variables",
       y = "Coefficient") +
  theme_minimal()
```

``` {r}
coefficients(boost_model)
```

``` {r}

#Pour SVM on affiche les coeff des variables 

variable_names <- names(coef(svm_model_linear))
coefficients <- coef(svm_model_linear)

svm_coefficients <- data.frame(
  Variables = variable_names,
  Coefficient = coefficients
)

#On enlève l'intercept pour le graphe
svm_coefficients <- svm_coefficients[svm_coefficients$Variables != "(Intercept)", ]
ggplot(svm_coefficients, aes(x = Variables, y = Coefficient)) +
  geom_bar(stat = "identity", fill = "skyblue", width = 0.5) +
  coord_flip() +
  labs(title = "Coeff pour le modèle SVM",
       x = "Variables",
       y = "Coefficient") +
  theme_minimal()
```

On voit également que c’est Electricty_from_renewables qui est la plus
importante.

``` {r}
#QUESTION 5

#Imputation des données manquantes par la moyenne de chaque variable

#On impute que pour les variables quantitatives (ici pas de pb puisque les 2 qualitatives Year et Entity n'avaient pas de données manquantes)
data_imputed <- data2 %>% 
                      mutate_if(is.numeric, ~ifelse(is.na(.), mean(., na.rm = TRUE), .))

#On enlève Electricity.from.fossil.fuels.TWh pour rester cohérent

data_imputed <- subset(data_imputed, select = -c(Electricity.from.fossil.fuels..TWh.))

#On vérifie qu'il n'y a plus de données manquantes
données_manquantes <- colSums(is.na(data_imputed))
print(données_manquantes)

fig <- plot_ly(x = names(données_manquantes), y = données_manquantes, type = "bar", 
               name = "Valeurs manquantes par variables", 
               marker = list(color = "purple"),
               xlab = "Variables", ylab = "Nombre de valeurs manquantes",
               title = "Valeurs manquantes par variables")

fig
```